# 第三章 刚体运动

> 所有坐标系遵循右手定则，即$\hat{x}\times\hat{y}=\hat{z}$

所有刚体位形都可以通过螺旋运动来实现，从而衍生出6参数位形表示法：**指数坐标（exponential coordinate）**。

- **运动旋量（twist）**或**空间速度（spatial velocity）**：3个角速度和3个线速度
- **力旋量（wrench）**或**空间力（spatial force）**：力和力矩


一些辅助函数：

In [48]:
import numpy as np # 导入numpy
from numpy import sin, cos

def Normalize(V):
    """Normalizes a vector

    :param V: A vector
    :return: A unit vector pointing in the same direction as z

    Example Input:
        V = np.array([1, 2, 3])
    Output:
        np.array([0.26726124, 0.53452248, 0.80178373])
    """
    return V / np.linalg.norm(V)


def NearZero(z):
    """Determines whether a scalar is small enough to be treated as zero

    :param z: A scalar input to check
    :return: True if z is close to zero, false otherwise

    Example Input:
        z = -1e-7
    Output:
        True
    """
    return abs(z) < 1e-6

## 3.1 平面内的刚体运动

**螺旋运动（screw motion）**表示：螺旋轴(screw axis) + 旋转角度

螺旋轴$S=(\omega, v_x, v_y)$, $\omega$为单位角速度, $v_x$和$v_y$为移动速度，旋转角度$\theta$，刚体位移$S\theta$，

角速度与线速度组合称为**运动旋量（twist）**，单位螺旋轴$S=(\omega, v_x, v_y)(\omega=1)$，旋转速度$\dot\theta$，则运动旋量$\mathcal{V} = S\dot\theta$


> 常见的机器人姿态表示方法：轴角法、欧拉角法，RPY角法，凯莱-罗德里格斯参数法，单位四元数法

**查理-莫兹（Chasles-Mozzi）定理**：任一刚体运动都可以通过绕某一螺旋轴旋转一定角度和平移来实现

## 3.2 旋转与角速度

### 旋转矩阵

旋转矩阵中的3个列向量分别对应物体坐标系的3个单位坐标轴，即$(\hat x_b, \hat y_b, \hat z_b)$

3×3的旋转矩阵组成的集合成为特殊正交群（special orthogonal group）$SO(3)$，满足：(1)$R^TR=I$; (2)$det R = 1$

> $SO(n)$群被称为矩阵李群（matrix Lie group），群中元素构成一个微分流形

旋转矩阵特性：

- **封闭性**：$AB$也是该群的一个元素
- **结合律**：$(AB)C=A(BC)$
- **幺元律**：存在单位元素$I$，满足$AI=IA=A$
- **可逆性**：存在可逆单元$A^{-1}$，满足$AA^{-1}=A^{-1}A=I$


旋转矩阵的应用：

- **表示姿态**： $R_{bc}$表示{c}相对于{b}的姿态
- **进行坐标系转换**，通过向量或坐标系来表示：下标消减原则，$R_{ab}R_{bc}=R_{ac}$
- **对向量或坐标系进行旋转变换**：$R_{sc}$既可以看成{c}在{s}中姿态，可以看做{s}到{c}的变换

旋转矩阵左乘和右乘：

- 左乘$R=Rot(\hat{\omega},\theta)$绕固定系中转轴$\hat{\omega}$转动
- 右乘$R=Rot(\hat{\omega},\theta)$绕物体系中转轴$\hat{\omega}$转动

> 个人理解：绕固定系转轴旋转，可以理解成{s}变换到{s'}，{b}在{s'}下的表达，因此$R_{sb'} = R_{s'b} = R_{s's}R_{sb'}$；绕相对系转轴旋转，可以理解成先将{s}变换到{b}，然后将{b}变换到{b''}，因此$R_{sb''} = R_{sb}R_{bb''}$。因此导致了左乘和右乘的区别（旋转轴不变）

定义旋转函数Rot(omega, theta)

In [49]:
def Rot(omega, theta):
    """General Rotation function

    :param omega: Axis 3x1 vector
    :param theta: Angle scalar
    :return: Rotation Matrix 3x3

    """
    omega = Normalize(omega)
    return np.array([[cos(theta)+omega[0]**2*(1-cos(theta)),                   omega[0]*omega[1]*(1-cos(theta))-omega[2]*sin(theta),   omega[0]*omega[2]*(1-cos(theta))+omega[1]*sin(theta)],
                     [omega[0]*omega[1]*(1-cos(theta))+omega[2]*sin(theta),    cos(theta)+omega[1]**2*(1-cos(theta)),                  omega[1]*omega[2]*(1-cos(theta))-omega[0]*sin(theta)],
                     [omega[0]*omega[2]*(1-cos(theta))-omega[1]*sin(theta),    omega[1]*omega[2]*(1-cos(theta))+omega[0]*sin(theta),   cos(theta)+omega[2]**2*(1-cos(theta))               ]])

测试一下旋转函数返回矩阵是否正确

In [50]:
print(Rot([1, 0, 0], 0.0)) #不旋转单位阵

A = Rot([0, 0, 1], 90.0/180.0*np.pi) #绕z旋转90°
print(A.dot(np.array([1,0,0])))


[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[6.123234e-17 1.000000e+00 0.000000e+00]


> 要旋转速度矢量$v$，只涉及用于表达$v$的坐标系，因此，$\hat\omega$也要在同一坐标系中描述，这个转动后的向量$v'$可写成$v'=Rv$

### 角速度

$\hat\omega$为瞬时单位转动轴，$\dot\theta$表示角速度，其组合表示**角速度（angular velocity）**（这个角速度$\omega$是矢量）
$$
\omega=\hat\omega\dot\theta
$$

在各个轴的速度分量：
$$
\dot{\hat x}=\omega\times\hat x \\
\dot{\hat y}=\omega\times\hat y \\
\dot{\hat z}=\omega\times\hat z
$$

在某一特定时刻$t$，令$\omega_s\in\mathbb{R}^3$表示固定坐标系中的角速度$\omega$，则姿态矩阵$R(t)$的时间变化率$\dot R(t)$表示成矩阵形式：
$$
\dot R(t) = \begin{bmatrix}\omega_s\times r_1 & \omega_s\times r_2 & \omega_s\times r_3 \end{bmatrix} = \omega_s \times R
$$

为了将叉积消去，定义一种新的运算$[\omega_s]R$，其中$[\omega_s]$ 为向量 $\omega_s\in\mathbb{R^3}$ 对应的**3×3反对称（skew-symmetric）矩阵**。
$$
[x] = \begin{bmatrix}
        0    & -x_3 &  x_2 \\
        x_3  &   0  & -x_1 \\
        -x_2 &  x_1 &   0
      \end{bmatrix}
$$

矩阵$[x]$就是与向量$x$相对应的3×3反对称矩阵。满足
$$
[x] = -[x]^T
$$

所有3×3反对称矩阵的集合称为so(3)，是**SO(3)群的李代数（Lie Algebra）**

> $\omega=\hat\omega\theta$就是SO(3)的指数坐标

In [51]:
def AxisAng3(expc3):
    """Converts a 3-vector of exponential coordinates for rotation into
    axis-angle form

    :param expc3: A 3-vector of exponential coordinates for rotation
    :return omghat: A unit rotation axis
    :return theta: The corresponding rotation angle

    Example Input:
        expc3 = np.array([1, 2, 3])
    Output:
        (np.array([0.26726124, 0.53452248, 0.80178373]), 3.7416573867739413)
    """
    return (Normalize(expc3), np.linalg.norm(expc3))

**旋转矩阵和反对称矩阵特性：**

给定任意$\omega\in\mathbb{R^3}$和$R\in SO(3)$，总能满足
$$
R[\omega]R^T=[R\omega]
$$

基于反对称矩阵定义，姿态矩阵变化率可重写为：
$$
[\omega_s]R=\dot R
[\omega_s] = \dot RR^{-1}
$$

$\omega_s$和$\omega_b$分别表示同一角速度在不同坐标系的表达：
$$
\omega_b=R_{sb}^{-1}\omega_s=R^{-1}\omega_s=R^T\omega_s
$$

令$R(t)$表示物体坐标系相对固定坐标系的姿态矩阵，则旋转体的角速度
$$
\dot RR^{-1}=[\omega_s] \\
R^{-1}\dot R=[\omega_b]
$$

> $\omega_b$并不是相对动坐标系的角速度，$\omega_b$表示的只是相对静坐标系{b}的角速度，**{b}只是与运动刚体随动坐标系瞬时重合**

In [52]:
def VecToso3(omega):
    """Converts a 3-vector to an so(3) representation

    :param omega: A 3-vector
    :return: The skew symmetric representation of omg

    Example Input:
        omg = np.array([1, 2, 3])
    Output:
        np.array([[ 0, -3,  2],
                  [ 3,  0, -1],
                  [-2,  1,  0]])
    """

    return np.array([[        0, -omega[2],  omega[1]],
                     [ omega[2],         0, -omega[0]],
                     [-omega[1],  omega[0],        0]])

In [53]:
def so3ToVec(so3mat):
    """Converts an so(3) representation to a 3-vector

    :param so3mat: A 3x3 skew-symmetric matrix
    :return: The 3-vector corresponding to so3mat

    Example Input:
        so3mat = np.array([[ 0, -3,  2],
                           [ 3,  0, -1],
                           [-2,  1,  0]])
    Output:
        np.array([1, 2, 3])
    """
    return np.array([so3mat[2][1], so3mat[0][2], so3mat[1][0]])

In [54]:
so3mat = VecToso3([1, 2, 3])
print(so3mat)

so3vec = so3ToVec(so3mat)
print(so3vec)


[[ 0 -3  2]
 [ 3  0 -1]
 [-2  1  0]]
[1 2 3]


### 转动的指数坐标表示

> 向量$\hat\omega\theta\in\mathbb{R}^3$就是**转动的三参数指数坐标**表示形式，单独写$\hat\omega$和$\theta$就是转动的轴-角(axis-angle)表示法。

**线性微分方程理论概述**

向量线性常微分方程：
$$
\dot x(t)=Ax(t)
$$
式中，$x(t)\in\mathbb{R}^n$, $A\in\mathbb{R}^{n\times n}$为常数，初始条件$x(0)=x_0$已知，解为：
$$
x(t)=e^{At}x_0
$$

**矩阵指数A**$e^{At}$满足如下特性：
- $d(e^{At})/dt=Ae^{At}=e^{At}A$
- $Ae^{At}=e^{At}A$
- 若$A=PDP^{-1}$（$D\in\mathbb{R}^{n\times n}$，可逆阵$P\in\mathbb{R}^{n\times n}$），则有$e^{At}=Pe^{Dt}P^{-1}$
- 若$AB=BA$，则有$e^Ae^B=e^{A+B}$
- $(e^A)^{-1}=e^{-A}$
$$

**罗德里格斯公式(Rodrigues' formula)：** 给定向量$\hat\omega\theta\in\mathbb{R}^3$，$\theta$为任一标量，而$\hat\omega\in\mathbb{R}^3$为单位向量，$[\hat\omega]\theta=[\hat\omega\theta]\in so(3)$的矩阵指数为
$$
\text{Rot}(\hat\omega, \theta)=e^{[\hat\omega]\theta}=I+\sin\theta[\hat\omega]+(1-\cos\theta)[\hat\omega]^2\in SO(3)
$$


In [55]:
def Rodrigues(omega, theta):
    """Rodrigues' formula implementation

    :param omega: A 3-Vector array
    :param theta: A scalar
    :return: The Rotaion Matrix in SO(3)
    """
    omega = Normalize(omega)
    return np.identity(3) + np.sin(theta)*VecToso3(omega) + (1 - np.cos(theta))* np.linalg.matrix_power(VecToso3(omega), 2)

In [56]:
# print(np.identity(3))
# print(np.eye(3))

A = Rodrigues(Normalize([1, 1, 0]), 90.0/180.0*np.pi)
# print("A", A)
# AN = Rodrigues([-1,-1,0], -90.0/180.0*np.pi)
# print("AN", AN)

print(np.dot(A.T, A))

B = Rot(Normalize([1, 1, 0]), 90.0/180.0*np.pi)
# print("B", B)
# BN = Rot([-1, -1, 0], -90.0/180.0*np.pi)
# print("BN", BN)

print(np.dot(B.T, B))

[[ 1.00000000e+00 -6.83580866e-17  0.00000000e+00]
 [-6.83580866e-17  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 1.00000000e+00  4.26642159e-17  1.22133484e-17]
 [ 4.26642159e-17  1.00000000e+00 -3.52068201e-17]
 [ 1.22133484e-17 -3.52068201e-17  1.00000000e+00]]


> **注意**：输入Rodrigues和Rot的函数一定要先Normalize，这个处理可以日后加到函数里，给定任一向量直接Normalize后再运算

**刚体转动的矩阵对数**

若$\hat\omega\theta\in\mathbb{R}^3$表示旋转矩阵$R$的指数坐标，那么反对称矩阵$[\hat\omega\theta]=[\hat\omega]\theta$就是矩阵$R$的**矩阵对数（matrix logarithm）**

> 矩阵指数是$e^{[\hat\omega]\theta}$，代表的是旋转矩阵$R\in\mathbb{R}^3$
> 
> 矩阵对数是$[\hat\omega]\theta$，代表的是旋转矩阵变化率（速度）

矩阵指数SO(3)和矩阵对数so(3)之间的转换函数

In [57]:
def MatrixExp3(so3mat):
    """Computes the matrix exponential of a matrix in so(3)

    :param so3mat: A 3x3 skew-symmetric matrix
    :return: The matrix exponential of so3mat

    Example Input:
        so3mat = np.array([[ 0, -3,  2],
                           [ 3,  0, -1],
                           [-2,  1,  0]])
    Output:
        np.array([[-0.69492056,  0.71352099,  0.08929286],
                  [-0.19200697, -0.30378504,  0.93319235],
                  [ 0.69297817,  0.6313497 ,  0.34810748]])
    """
    omgtheta = so3ToVec(so3mat) # 从so(3)获取向量\omega=\hat\omega\theta
    if NearZero(np.linalg.norm(omgtheta)):
        return np.eye(3) # 如果omgtheta为零向量，则说明没转动，直接返回单位阵
    else:
        theta = np.linalg.norm(omgtheta) # 获取theta值，就是L2范数
        omega = omgtheta / theta         # 获取单位旋转轴\hat\omega
        return Rodrigues(omega, theta)   # Rodrigues公式获取旋转矩阵SO(3)

In [58]:
def MatrixLog3(R):
    """Computes the matrix logarithm of a rotation matrix

    :param R: A 3x3 rotation matrix
    :return: The matrix logarithm of R

    Example Input:
        R = np.array([[0, 0, 1],
                      [1, 0, 0],
                      [0, 1, 0]])
    Output:
        np.array([[          0, -1.20919958,  1.20919958],
                  [ 1.20919958,           0, -1.20919958],
                  [-1.20919958,  1.20919958,           0]])
    """

    cos_theta = (np.trace(R) - 1.0) / 2.0 # 获取cosθ的值
    if cos_theta >= 1: # 对应R=I, θ=0的情况，此时没转动，ω不存在，直接返回零矩阵
        np.zeros(3, 3)
    elif cos_theta <= -1: # 对应θ=π的情况，此时为以下三种中任一值（主要看分母是否为0）
        if not NearZero(1 + R[2][2]):
            omega = (1.0 / np.sqrt(2 * (1 + R[2][2]))) * np.array([R[0][2], R[1][2], 1 + R[2][2]])
        elif not NearZero(1 + R[1][1]):
            omega = (1.0 / np.sqrt(2 * (1 + R[1][1]))) * np.array([R[0][1], 1 + R[1][1], R[2][1]])
        else:
            omega = (1.0 / np.sqrt(2 * (1 + R[0][0]))) * np.array([1 + R[0][0], R[1][0], R[2][0]])
        return VecToso3(np.pi * omega)
    else: # 其他情况
        theta = np.arccos(cos_theta)
        return theta / 2.0 / np.sin(theta) * (R - np.array(R).T)

测试：

In [76]:
vec = [1, 2, 1] # 这个就是指数坐标
so3mat = VecToso3(vec) # 获取so3mat，这个就是矩阵对数

print("Axis Angle: \n ", AxisAng3(vec))

# so3mat = np.array([[ 0, -3,  2],
#                     [ 3,  0, -1],
#                     [-2,  1,  0]])

print("so3mat:\n ", so3mat)

R = MatrixExp3(so3mat)

print("MatrixExp3：\n", R)

# print("R:\n", Rodrigues(*AxisAng3(vec)))

Log3 = MatrixLog3(R);

print("MatrixLog3:\n", Log3)

so3vec = so3ToVec(Log3)
print("so3 vec: ", so3vec)

print("Axis Angle: \n ", AxisAng3(so3vec)) # 这个结果和之前的vec等价，相当于正反转到2pi

Axis Angle: 
  (array([0.40824829, 0.81649658, 0.40824829]), 2.449489742783178)
so3mat:
  [[ 0 -1  2]
 [ 1  0 -1]
 [-2  1  0]]
MatrixExp3：
 [[-0.47492144  0.32944181  0.81603782]
 [ 0.85049534  0.41003142  0.32944181]
 [-0.22606924  0.85049534 -0.47492144]]
MatrixLog3:
 [[ 0. -1.  2.]
 [ 1.  0. -1.]
 [-2.  1.  0.]]
so3 vec:  [1. 2. 1.]
Axis Angle: 
  (array([0.40824829, 0.81649658, 0.40824829]), 2.4494897427831814)


$$
[\hat\omega]=\frac{1}{2\sin\theta}(R-R^T)
$$

因此当$\theta$是$\pi$的整数倍，$[\hat\omega]$不能很好定义

> 对于转动的任意三参数表示，类似的奇异是不可避免的

**算法：** 给定$R\in SO)=(3)$，总是能找到单位转轴$\hat\omega\in\mathbb{R}^3$，且$||\hat\omega||=1$，$\theta\in(0,\pi)$，使得$R=e^{[\hat\omega]\theta}$。其中，向量$\hat\omega\theta\in\mathbb{R}^3$是$R$的指数坐标，而反对称矩阵$[\hat\omega]\in so(3)$是$R$的矩阵对数。

- 若$R=I$，则$\theta=0$，而$\hat\omega$不确定。（相当于根本没转动）
- 若$\text{tr}R=-1$，则$\theta=\pi$。这时，$\hat\omega$可以取下述3种情况的任一值。

$$
\hat\omega=\frac{1}{\sqrt{2(1+r_{33})}}\begin{bmatrix}r13\\r23\\1+r_{33}\end{bmatrix}
$$
或者
$$
\hat\omega=\frac{1}{\sqrt{2(1+r_{22})}}\begin{bmatrix}r12\\1+r22\\r_{32}\end{bmatrix}
$$
或者
$$
\hat\omega=\frac{1}{\sqrt{2(1+r_{11})}}\begin{bmatrix}1+r11\\r21\\r_{31}\end{bmatrix}
$$
（注意到$-\hat\omega$也可以作为一组解）

其他情况下，
$$
\theta=\cos^{-1}(\frac 1 2 (\text{tr}R-1))\in [0,\pi)] \newline

 [\hat\omega]=\frac{1}{2\sin\theta}(R-R^T)
$$

> 旋转群SO(3)可以看成一个半径为$\pi$的实体球，指数坐标$r=\hat\omega\theta$可以位于实体球内的任意位置。

## 刚体运动与运动旋量

- 齐次变换矩阵$T$与旋转矩阵$R$相对应
- 螺旋轴$S$与旋转轴$\hat\omega$相对应
- 运动旋量$\mathcal{V}(S\dot\theta)$与角速度$\omega=\hat\omega\dot\theta$相对应
- 刚体位移指数坐标$S\theta\in\mathbb{R}^6$与刚体转动的指数坐标$\hat\omega\theta\in\mathbb{R}^3$

**齐次变换矩阵**：特殊欧式群（special Euclidean group）SE(3)称为刚体运动运动（rigid-body motion）群或齐次变换矩阵（homogeneous transformation matrice）群，是所有4×4实矩阵$T$的集合，可写成：
$$
T=\begin{bmatrix} R & p \\ 0 & 1 \end{bmatrix} = \begin{bmatrix} r_{11} & r_{12} & r_{13} & p_{1} \\ r_{21} & r_{22} & r_{23} & p_{2} \\ r_{31} & r_{32} & r_{33} & p_{3} \\ 0 & 0 & 0 & 1 \end{bmatrix}
$$

式中，$R\in SO(3)$，$p\in\mathbb{R}^3$为列向量

齐次变换矩阵特性：

- 齐次变换矩阵$T\in SE(3)$的逆矩阵也是齐次变换矩阵，可以写成：
$$
T^{-1}=\begin{bmatrix}R & p \\ 0 & 1\end{bmatrix}^{-1}=\begin{bmatrix}R^T & -R^Tp \\ 0 & 1\end{bmatrix}
$$
- 两个齐次变换矩阵的乘积也是齐次变换矩阵
- 齐次变换矩阵的乘法满足结合律$(T_1T_2)T_3=T_1(T_2T_3)$，但一般不满足交换律$T_1T_2\neq T_2T_1$

**齐次坐标（homogeneous coordinate）：** 向量$[x^T \quad 1]^T$

满足：
- $||Tx-Ty||=||x-y||$，$||\cdot||$表示向量在$\mathbb{R}^3$中标准范数，$||x||=\sqrt{x^Tx}$，也就是距离不变，**等距特性（preserves distances）**
- $<Tx-Tz, Ty-Tz>=<x-z, y-z>$，式中$<\cdot, \cdot>$表示向量标准內积，$<x,y>=x^Ty$，**保角特性（preserves angles）**


齐次变换矩阵用途：
- 表示刚体的位形（位置和姿态）
- 变换参考坐标系（用向量或坐标系来表示）
- 表示向量或坐标系的位移


**平移和旋转顺序**

- 左乘$T$，固定坐标系变换，先旋转$R$，再平移$p$
- 右乘$T$，物体坐标系变换，先平移$p$，再旋转$R$

### 运动旋量 Twist

- **物体运动旋量（body twist）**，定义为**物体坐标系中的速度**
$$
\mathcal{V}_b=\begin{bmatrix}\omega_b \\ v_b \end{bmatrix}\in\mathbb{R}^6, 
[\mathcal{V}_b]=\begin{bmatrix}[\omega_b] & v_b \\ 0 & 0\end{bmatrix}=T^{-1}\dot T\in se(3)
$$

- **空间运动旋量（spatial twist）**，定义为**空间固定坐标系中的速度**
$$
\mathcal{V}_s=\begin{bmatrix}\omega_s \\ v_s \end{bmatrix}\in\mathbb{R}^6, 
[\mathcal{V}_s]=\begin{bmatrix}[\omega_s] & v_s \\ 0 & 0\end{bmatrix}=\dot TT^{-1}\in se(3)
$$

> $v_s$可以看成刚体上与固定坐标系原点相重合的点的瞬时速度，并在固定坐标系中的度量

$$
[\mathcal{V}_b]=T^{-1}[\mathcal{V}_s]T \newline
[\mathcal{V}_s]=T[\mathcal{V}_b]T^{-1} 
$$


$\mathcal{V}_b$与$\mathcal{V}_s$之间关系：
$$
\begin{bmatrix}\omega_s \\ v_s\end{bmatrix}=\begin{bmatrix}R & 0 \\ [p]R & R\end{bmatrix}\begin{bmatrix}\omega_b \\ v_b\end{bmatrix}
$$

给定$T=(R,p)\in SE(3)$，其伴随变换矩阵(adjoint representation)$[Ad_T]$为：
$$
[Ad_T]=\begin{bmatrix}R & 0 \\ [p]R & R\end{bmatrix}\in\mathbb{R}^{6\times 6}
$$


- $\mathcal{V}'=[Ad_T]\mathcal{V}$
- $\mathcal{V}'=Ad_T(\mathcal{V})$
- $[\mathcal{V}']=T[\mathcal{V}]T^{-1}$

伴随矩阵性质：
- $[Ad_{T_1}][Ad_{T_2}]\mathcal{V}=[Ad_{T_1T_2}]\mathcal{V}$
- $[Ad_T]^{-1}=[Ad_{T^{-1}}]$

TransToRp()：将齐次坐标拆分成旋转矩阵和位置向量

In [60]:
def TransToRp(T):
    """Converts a homogeneous transformation matrix into a rotation matrix
    and position vector

    :param T: A homogeneous transformation matrix
    :return R: The corresponding rotation matrix,
    :return p: The corresponding position vector.

    Example Input:
        T = np.array([[1, 0,  0, 0],
                      [0, 0, -1, 0],
                      [0, 1,  0, 3],
                      [0, 0,  0, 1]])
    Output:
        (np.array([[1, 0,  0],
                   [0, 0, -1],
                   [0, 1,  0]]),
         np.array([0, 0, 3]))
    """
    T = np.array(T)
    return T[0: 3, 0: 3], T[0: 3, 3]

In [61]:
def RpToTrans(R, p):
    """Converts a rotation matrix and a position vector into homogeneous
    transformation matrix

    :param R: A 3x3 rotation matrix
    :param p: A 3-vector
    :return: A homogeneous transformation matrix corresponding to the inputs

    Example Input:
        R = np.array([[1, 0,  0],
                      [0, 0, -1],
                      [0, 1,  0]])
        p = np.array([1, 2, 5])
    Output:
        np.array([[1, 0,  0, 1],
                  [0, 0, -1, 2],
                  [0, 1,  0, 5],
                  [0, 0,  0, 1]])
    """
    return np.r_[np.c_[R, p], [[0, 0, 0, 1]]]

TransInv()：齐次矩阵求逆，主要基于$T^{-1}=\begin{bmatrix}R & p \\ 0 & 1\end{bmatrix}^{-1}=\begin{bmatrix}R^T & -R^Tp \\ 0 & 1\end{bmatrix}
$

In [62]:
def TransInv(T):
    """Inverts a homogeneous transformation matrix

    :param T: A homogeneous transformation matrix
    :return: The inverse of T
    Uses the structure of transformation matrices to avoid taking a matrix
    inverse, for efficiency.

    Example input:
        T = np.array([[1, 0,  0, 0],
                      [0, 0, -1, 0],
                      [0, 1,  0, 3],
                      [0, 0,  0, 1]])
    Output:
        np.array([[1,  0, 0,  0],
                  [0,  0, 1, -3],
                  [0, -1, 0,  0],
                  [0,  0, 0,  1]])
    """
    R, p = TransToRp(T)
    Rt = np.array(R).T
    return np.r_[np.c_[Rt, -np.dot(Rt, p)], [[0, 0, 0, 1]]]

Adjoint()：获取齐次矩阵的伴随矩阵

In [63]:
def Adjoint(T):
    """Computes the adjoint representation of a homogeneous transformation
    matrix

    :param T: A homogeneous transformation matrix
    :return: The 6x6 adjoint representation [AdT] of T

    Example Input:
        T = np.array([[1, 0,  0, 0],
                      [0, 0, -1, 0],
                      [0, 1,  0, 3],
                      [0, 0,  0, 1]])
    Output:
        np.array([[1, 0,  0, 0, 0,  0],
                  [0, 0, -1, 0, 0,  0],
                  [0, 1,  0, 0, 0,  0],
                  [0, 0,  3, 1, 0,  0],
                  [3, 0,  0, 0, 0, -1],
                  [0, 0,  0, 0, 1,  0]])
    """

    R, p = TransToRp(T)

    return np.r_[np.c_[R, np.zeros([3,3])], 
                 np.c_[np.dot(VecToso3(p), R), R]];


例题 3.23

In [64]:
v_s = np.array([0, 0, 2, -2, -4, 0])
v_b = np.array([0, 0, -2, 2.8, 4, 0])

T_sb = np.array([[ -1,  0,  0,    4],
                 [  0,  1,  0,  0.4],
                 [  0,  0, -1,    0],
                 [  0,  0,  0,    1]])


print(v_s)
print(np.dot(Adjoint(T_sb), v_b))

print(v_b)
print(np.dot(Adjoint(TransInv(T_sb)), v_s))


[ 0  0  2 -2 -4  0]
[ 0.  0.  2. -2. -4.  0.]
[ 0.   0.  -2.   2.8  4.   0. ]
[ 0.   0.  -2.   2.8  4.   0. ]


**运动旋量的螺旋释义**

- 若$\omega\neq 0$，$\mathcal{S}=\mathcal{V}/||\omega||=(\omega/||\omega||, v/||\omega||)$，螺旋轴角速度$\dot\theta=||\omega||$，$\mathcal{S}\dot\theta=\mathcal{V}$
- 若$\omega = 0$，$\mathcal{S}=\mathcal{V}/||v||=(0, v/||v||)$，螺旋轴角速度$\dot\theta=||v||$，$\mathcal{S}\dot\theta=\mathcal{V}$

> 螺旋轴$\mathcal{S}$和$\mathcal{V}$的区别：螺旋轴$\mathcal{S}$中$||\omega||$或$||v||$必为1，而旋量$\mathcal{V}$则对$\omega$和$v$没限制

给定参考系，螺旋轴$\mathcal{S}$可写成
$$
\mathcal{S}=\begin{bmatrix}\omega \\ v\end{bmatrix}\in \mathbb{R}^6
$$

式中:
- 若$||\omega||=1$，则$v=-\omega\times q + h\omega$，式中$q$为**轴**上任一点，$h$为螺旋的节距（纯转动则$h=0$）
- 若$\omega=0, ||v||=1$， 则螺旋的节距无穷大，对应的运动旋量为纯转动，参数中只有$v$

> 这个很关键，是在正运动学中求解旋量的方法

### 刚体运动的指数坐标表达

$$
\text{exp: } [\mathcal{S}\theta]\in se(3) \rightarrow T\in SE(3) \newline
\text{log: } T\in SE(3) \rightarrow [\mathcal{S}\theta]\in se(3)
$$

令$\mathcal{S}=(\omega, v)$为螺旋轴，若$||\omega||=1$， 则对于任意沿螺旋轴的距离$\theta\in\mathbb{R}$，都有
$$
 e^{[\mathcal{S}]\theta}=\begin{bmatrix}e^{[\omega]\theta} & (I\theta+(1-\cos\theta)[\omega]+(\theta-\sin\theta)[\omega]^2)v \\ 0 & 1 \end{bmatrix}
$$

若$\omega=0$，$||v||=1$，则
$$
e^{[\mathcal{S}]\theta}=\begin{bmatrix}I & v\theta \\ 0 & 1\end{bmatrix}
$$

给定任意的$(R,p)\in SE(3)$，总能找到与之相对应的螺旋轴$\mathcal{S}=(\omega, v)$和标量$\theta$，满足
$$
e^{[\mathcal{S}]\theta}=\begin{bmatrix}R & p \\ 0 & 1\end{bmatrix}
$$
式中，矩阵
$$
[\mathcal{S}]\theta=\begin{bmatrix}[\omega]\theta & v\theta \\ 0 & 1\end{bmatrix}\in se(3)
$$
是$T=(R,p)$的矩阵对数形式。

## 力旋量 Wrench

作用在刚体上一点$r\in\mathbb{R}^3$的纯力$f\in\mathbb{R}^3$，该力在参考坐标系{a}产生的力矩（torque）或力偶（moment）$m\in\mathbb{R}^3$可写成：
$$
m = r\times f
$$


将力和力矩合成一个六维的**空间力（spatial force）**，称为**力旋量（wrench）**：
$$
\mathcal{F} = \begin{bmatrix}m \\ f\end{bmatrix}\in\mathbb{R}^6
$$

> 注意：力矩放在前面


力旋量$\mathcal{F}$及其在坐标系{a}中的表示$\mathcal{F}_a$和在坐标系{b}中的表示$\mathcal{F}_b$，两者之间关系可写成：
$$
\mathcal{F}_b=Ad_{T_{ab}}^T(\mathcal{F}_a)=[Ad_{T_{ab}}]^T\mathcal{F}_a \newline
\mathcal{F}_a=Ad_{T_{ba}}^T(\mathcal{F}_b)=[Ad_{T_{ba}}]^T\mathcal{F}_b
$$

例题 3.28 思路：

因为需要得到六维力传感器处的力旋量$\mathcal{F}_f$，得到机械手重力在其质心处的力旋量$\mathcal{F}_h$（没有力矩作用，只有力），以及苹果重力在其质心处的力旋量$\mathcal{F}_a$，则$\mathcal{F}_f = [Ad_{T_{hf}}]^T\mathcal{F}_h + [Ad_{T_{af}}]^T\mathcal{F}_a $

In [65]:
L_1 = 0.1 # 10 cm
L_2 = 0.15 # 15 cm

F_h = [0, 0, 0, 0, -5, 0]
F_a = [0, 0, 0, 0,  0, 1]

p_hf = [-L_1, 0, 0]
p_af = [-(L_1 + L_2), 0, 0]

R_hf = Rodrigues([0, 0, 1], 0) # 方向一致
R_af = Rodrigues([1, 0, 0], -90.0 / 180.0 * np.pi) # 绕x_a转-90°

T_hf = RpToTrans(R_hf, p_hf)
T_af = RpToTrans(R_af, p_af)

F_f = np.dot(Adjoint(T_hf).T, F_h) + np.dot(Adjoint(T_af).T, F_a)
F_f


array([ 0.00000000e+00, -2.77555756e-17, -7.50000000e-01,  0.00000000e+00,
       -6.00000000e+00,  1.11022302e-16])